In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sys
import p5lib

%matplotlib inline

In [2]:
data = pd.read_csv('MERGED2013_PP.csv')
print "Number of features: {}".format(len(data.columns))
print "Number of rows: {}".format(len(data))

Number of features: 1729
Number of rows: 7804


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data = p5lib.preprocess_data(data)
print data.shape

(6007, 42)


In [4]:
X = data[['CONTROL', 'DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'PAR_ED_PCT_1STGEN', 
          'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 'UGDS', 
          'WDRAW_DEBT_MDN', 'L4_COLLEGE', 'NPT4', 'NUM4', 
          'PFTFTUG1_EF', 'PFTFAC',
          'SAT_AVG_ALL', 'ACTCMMID', 'ADM_RATE_ALL', 'AVGFACSAL', 'COSTT4_A',
          'C150', 'RET_FT']]
y = X[['C150', 'RET_FT']]
X = X.drop('C150', 1)
X = X.drop('RET_FT', 1)
print X.shape
print y.shape


(6007, 25)
(6007, 2)


In [5]:
p5lib.print_num_data_for_each_features(X)

Number of available data for each feature (not counting the NaN values)
CONTROL             Control (public/private)                      6007
DEBT_MDN            Median debt                                   5242
DEP_INC_AVG         Avg income dependent stu                      5703
GRAD_DEBT_MDN       Median debt complete                          5186
IND_INC_AVG         Avg income independent stu                    5688
INEXPFTE            Expense per FTE student                       6006
PAR_ED_PCT_1STGEN   % 1st gen students                            5513
PAR_ED_PCT_HS       % parent education high school                5533
PAR_ED_PCT_MS       % parent education middle school              5423
PAR_ED_PCT_PS       % parent education post secondary             5533
PCTFLOAN            % Fed student loan                            6005
PCTPELL             % Pell Grant receiver                         6005
UG25abv             % undergrad > 25 yr                           5964
UGDS 

In [6]:
fill_cols_with_mean = ['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'WDRAW_DEBT_MDN',
                       'PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 
                       'UG25abv', 'NPT4', 'NUM4', 'PFTFTUG1_EF', 'PFTFAC', 'SAT_AVG_ALL', 'ACTCMMID', 'ADM_RATE_ALL', 
                       'AVGFACSAL', 'COSTT4_A']
for col in fill_cols_with_mean:
    X[col] = X[col].fillna(X[col].mean())

In [7]:
p5lib.print_num_data_for_each_features(X)

Number of available data for each feature (not counting the NaN values)
CONTROL             Control (public/private)                      6007
DEBT_MDN            Median debt                                   6007
DEP_INC_AVG         Avg income dependent stu                      6007
GRAD_DEBT_MDN       Median debt complete                          6007
IND_INC_AVG         Avg income independent stu                    6007
INEXPFTE            Expense per FTE student                       6007
PAR_ED_PCT_1STGEN   % 1st gen students                            6007
PAR_ED_PCT_HS       % parent education high school                6007
PAR_ED_PCT_MS       % parent education middle school              6007
PAR_ED_PCT_PS       % parent education post secondary             6007
PCTFLOAN            % Fed student loan                            6007
PCTPELL             % Pell Grant receiver                         6007
UG25abv             % undergrad > 25 yr                           6007
UGDS 

In [8]:
from sklearn import preprocessing

tmpX = X[['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'UGDS', 'WDRAW_DEBT_MDN', 'NPT4', 
          'NUM4', 'SAT_AVG_ALL', 'ACTCMMID', 'AVGFACSAL', 'COSTT4_A']]

scaler = preprocessing.MinMaxScaler()
scaledX = scaler.fit_transform(tmpX)

# print np.array(tmpX.ix[0])
# print scaledX[0]

# X_for_PCA = pd.DataFrame(scaledX)
pctX = X[['PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 
          'PFTFTUG1_EF', 'PFTFAC', 'ADM_RATE_ALL']]
pctX = np.array(pctX)

forPcaX = np.concatenate((scaledX, pctX), axis=1)
# print scaledX.shape
# print pctX.shape
print forPcaX.shape


(6007, 23)


In [9]:
# add the categorical feature

tmp1X = np.array(pd.get_dummies(X['CONTROL']))
tmp2X = X['L4_COLLEGE'].astype(int)
tmp2X = np.reshape(tmp2X, (len(tmp2X), 1)) 
print tmp1X.shape
print tmp2X.shape
print forPcaX.shape
finalX = np.concatenate((forPcaX, tmp1X, tmp2X), axis=1)
print finalX.shape

(6007, 3)
(6007, 1)
(6007, 23)
(6007, 27)


In [10]:
from sklearn import cross_validation as cv

y = np.array(y)
X_train, X_test, y_train, y_test = cv.train_test_split(finalX, y, train_size=0.8)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(4805, 27)
(1202, 27)
(4805, 2)
(1202, 2)


In [11]:
SVR_reg1, SVR_reg2 = p5lib.build_SVR_model(X_train, X_test, y_train, y_test)

{'epsilon': 0.10000000000000001, 'C': 10.0, 'gamma': 0.10000000000000001}
R2 score on train data: 0.697698073988
R2 score on test  data: 0.694721474137
{'epsilon': 0.10000000000000001, 'C': 10.0, 'gamma': 0.10000000000000001}
R2 score on train data: 0.388220378858
R2 score on test  data: 0.361304282415


In [12]:
DT_reg1, DT_reg2 = p5lib.build_DecisionTree_model(X_train, X_test, y_train, y_test)

{'max_depth': 8}
R2 score on train data: 0.72765000789
R2 score on test  data: 0.621747514601
{'max_depth': 4}
R2 score on train data: 0.268504122153
R2 score on test  data: 0.247468323036


In [13]:
KNN_reg1, KNN_reg2 = p5lib.build_KNN_model(X_train, X_test, y_train, y_test)

{'n_neighbors': 10}
R2 score on train data: 0.719433982924
R2 score on test  data: 0.671840279624
{'n_neighbors': 19}
R2 score on train data: 0.378931702457
R2 score on test  data: 0.312646965374


In [14]:
RForest_reg1, RForest_reg2 = p5lib.build_RandomForest_model(X_train, X_test, y_train, y_test)

R2 score on train data: 0.942934973302
R2 score on test  data: 0.691506279459
R2 score on train data: 0.868314495169
R2 score on test  data: 0.304831865657
